<a href="https://colab.research.google.com/github/Chienstartup/ISIC_2024/blob/main/Training_CNN_with_pretrained_structure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torcheval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 12.5 MB/s eta 0:00:00


In [2]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 63.3 MB/s eta 0:00:00


In [3]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.4 MB/s eta 0:00:00


In [4]:
!pip install torchmetrics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 866.2/866.2 kB 43.8 MB/s eta 0:00:00


In [5]:
import h5py
import io
from io import BytesIO
import PIL
from PIL import Image

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp
import torchvision
from torcheval.metrics.functional import binary_auroc

# Utils
import joblib
from tqdm import tqdm
from collections import defaultdict

# Sklearn Imports
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, StratifiedGroupKFold,GroupKFold

# For Image Models
import timm

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2



import warnings
warnings.filterwarnings("ignore")
import os
# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import gc
import random
import time
import copy
from collections import defaultdict
import cv2

from torchmetrics import AUROC

import optuna
from sklearn.utils import resample

In [6]:
balanced_df = pd.read_csv('https://raw.githubusercontent.com/Chienstartup/ISIC_2024/main/meta_data/df_train_filtered.csv')

In [7]:
CONFIG = {
    "seed": 42,
    "epochs": 30,
    "img_size": 384,
    "model_name": 'tf_efficientnetv2_b1.in1k',
    "train_batch_size": 64,
    "valid_batch_size": 128,
    "learning_rate": 1e-4,
    "scheduler": 'CosineAnnealingWarmRestarts',
    "min_lr": 1e-6,
    "T_max": 500,
    "weight_decay": 1e-6,
    "n_fold": 5,
    "n_accumulate": 1,
    "device": torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    "num_classes": 1,
    "T_0": 10,
    "T_mult": 1,
}

In [8]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(CONFIG['seed'])

<h2>For the hdf file, you need to upload it to colab by yourself due to GitHub constraint</h2>

In [10]:
train_hdf = h5py.File('/content/sample_5k.hdf5', 'r')

In [11]:
auroc = AUROC(task="binary", num_classes=1).to(CONFIG['device'])

In [12]:
class ISICDataset(Dataset):
    def __init__(self, df, file_hdf, transforms=None):
        self.df = df
        self.fp_hdf = file_hdf
        self.isic_ids = df['isic_id'].values
        self.targets = df['target'].values
        self.transforms = transforms

    def __len__(self):
        return len(self.isic_ids)

    def __getitem__(self, index):
        isic_id = self.isic_ids[index]
        img = np.array( Image.open(BytesIO(self.fp_hdf[isic_id][()])) )
        target = self.targets[index]

        if self.transforms:
            img = self.transforms(image=img)["image"]

        return {
            'image': img,
            'target': target,
        }

In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import timm

class GeM(nn.Module):
    def __init__(self, p=3, eps=1e-6):
        super(GeM, self).__init__()
        self.p = nn.Parameter(torch.ones(1) * p)
        self.eps = eps

    def forward(self, x):
        return self.gem(x, p=self.p, eps=self.eps)

    def gem(self, x, p=3, eps=1e-6):
        return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)

class ISICModel(nn.Module):
    def __init__(self, model_name, num_classes=1, pretrained=True, dropout_rate=0.5):
        super(ISICModel, self).__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, num_classes=0)

        if hasattr(self.model, 'num_features'):
            in_features = self.model.num_features
        elif hasattr(self.model, 'fc'):
            in_features = self.model.fc.in_features
        else:
            raise ValueError("Unable to determine the number of input features for the classifier")

        self.dropout = nn.Dropout(dropout_rate)
        self.classifier = nn.Linear(in_features, num_classes)

    def forward(self, x):
        features = self.model(x)
        x = self.dropout(features)
        return self.classifier(x)

In [14]:
# 損失函數
criterion = nn.BCEWithLogitsLoss()

In [15]:
def train_one_epoch(model, optimizer, scheduler, criterion, dataloader, device, epoch):
    model.train()
    dataset_size = 0
    running_loss = 0.0

    predictions = []
    targets = []

    for data in dataloader:
        images = data['image'].to(device, dtype=torch.float)
        target = data['target'].to(device, dtype=torch.float).view(-1, 1)

        batch_size = images.size(0)

        outputs = model(images)
        loss = criterion(outputs, target)
        loss = loss / CONFIG['n_accumulate']

        loss.backward()

        if (dataset_size + batch_size) % (CONFIG['n_accumulate'] * CONFIG['train_batch_size']) == 0:
            optimizer.step()
            optimizer.zero_grad()
            if scheduler is not None:
                scheduler.step()

        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size

        predictions.extend(torch.sigmoid(outputs).detach().cpu().numpy())
        targets.extend(target.detach().cpu().numpy())

    epoch_loss = running_loss / dataset_size

    predictions = np.concatenate(predictions)
    targets = np.concatenate(targets)
    epoch_metric = custom_metric_binary(targets, predictions)

    gc.collect()

    return epoch_loss, epoch_metric

In [16]:
@torch.inference_mode()
def valid_one_epoch(model, criterion, dataloader, device, epoch):
    model.eval()
    dataset_size = 0
    running_loss = 0.0

    predictions = []
    targets = []

    for data in dataloader:
        images = data['image'].to(device, dtype=torch.float)
        target = data['target'].to(device, dtype=torch.float).view(-1, 1)

        batch_size = images.size(0)
        outputs = model(images)
        loss = criterion(outputs, target)

        running_loss += (loss.item() * batch_size)
        dataset_size += batch_size

        predictions.extend(torch.sigmoid(outputs).cpu().numpy())
        targets.extend(target.cpu().numpy())

    epoch_loss = running_loss / dataset_size

    predictions = np.concatenate(predictions)
    targets = np.concatenate(targets)
    epoch_metric = custom_metric_binary(targets, predictions)

    gc.collect()

    return epoch_loss, epoch_metric

In [17]:
def run_training(model, optimizer, scheduler, criterion, device, num_epochs, train_loader, valid_loader):
    if torch.cuda.is_available():
        print("[INFO] Using GPU: {}\n".format(torch.cuda.get_device_name()))

    start = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_epoch_auroc = -np.inf
    history = defaultdict(list)

    for epoch in tqdm(range(1, num_epochs + 1), desc="Training Progress"):
        gc.collect()
        train_epoch_loss, train_epoch_auroc = train_one_epoch(model, optimizer, scheduler,
                                           criterion, train_loader, device, epoch)

        val_epoch_loss, val_epoch_auroc = valid_one_epoch(model, criterion, valid_loader, device, epoch)

        history['Train Loss'].append(train_epoch_loss)
        history['Valid Loss'].append(val_epoch_loss)
        history['Train AUROC Class 1'].append(train_epoch_auroc)
        history['Valid AUROC Class 1'].append(val_epoch_auroc)

        print(f'Epoch {epoch}/{num_epochs}:')
        print(f'Train Loss: {train_epoch_loss:.4f}, Train AUROC: {train_epoch_auroc:.4f}')
        print(f'Valid Loss: {val_epoch_loss:.4f}, Valid AUROC: {val_epoch_auroc:.4f}')

        if val_epoch_auroc > best_epoch_auroc:
            print(f"Validation AUROC for Class 1 Improved ({best_epoch_auroc:.4f} ---> {val_epoch_auroc:.4f})")
            best_epoch_auroc = val_epoch_auroc
            best_model_wts = copy.deepcopy(model.state_dict())
            print('Model Saved!')

        print()

    end = time.time()
    time_elapsed = end - start
    print('Training complete in {:.0f}h {:.0f}m {:.0f}s'.format(
        time_elapsed // 3600, (time_elapsed % 3600) // 60, (time_elapsed % 3600) % 60))
    print("Best AUROC for Class 1: {:.4f}".format(best_epoch_auroc))

    model.load_state_dict(best_model_wts)

    return model, history

In [18]:
def fetch_scheduler(optimizer):
    if CONFIG['scheduler'] == 'CosineAnnealingLR':
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer, T_max=CONFIG['T_max'],
                                                   eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] == 'CosineAnnealingWarmRestarts':
        scheduler = lr_scheduler.CosineAnnealingWarmRestarts(optimizer,
                                                             T_0=CONFIG['T_0'],
                                                             T_mult=CONFIG['T_mult'],
                                                             eta_min=CONFIG['min_lr'])
    elif CONFIG['scheduler'] is None:
        return None
    else:
        raise ValueError(f"Unsupported scheduler: {CONFIG['scheduler']}")
    return scheduler

In [20]:
import albumentations as A
from albumentations.pytorch import ToTensorV2

data_transforms = {
    "train": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.OneOf([
            A.RandomRotate90(p=0.5),
            A.Transpose(p=0.5),
            A.VerticalFlip(p=0.5),
            A.HorizontalFlip(p=0.5),
        ], p=0.5),
        A.OneOf([
            A.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2, p=0.5),
            A.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
        ], p=0.5),
        A.OneOf([
            A.MotionBlur(blur_limit=5, p=0.5),
            A.MedianBlur(blur_limit=5, p=0.5),
            A.GaussianBlur(blur_limit=5, p=0.5),
        ], p=0.3),
        A.GaussNoise(var_limit=(5.0, 30.0), p=0.3),
        A.OneOf([
            A.OpticalDistortion(distort_limit=1.0, p=0.5),
            A.GridDistortion(num_steps=5, distort_limit=0.5, p=0.5),
            A.ElasticTransform(alpha=1, sigma=50, alpha_affine=None, p=0.5),
        ], p=0.3),
        A.CLAHE(clip_limit=4.0, p=0.3),
        A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.5),
        A.CoarseDropout(max_holes=8, max_height=CONFIG['img_size']//20, max_width=CONFIG['img_size']//20, p=0.3),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
        ToTensorV2()
    ], p=1.),

    "valid": A.Compose([
        A.Resize(CONFIG['img_size'], CONFIG['img_size']),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
        ToTensorV2()
    ], p=1.)
}


In [21]:
def cutmix(batch, alpha=1.0):
    images = batch['image']
    targets = batch['target']

    batch_size = images.size(0)
    indices = torch.randperm(batch_size)
    shuffled_images = images[indices]
    shuffled_targets = targets[indices]

    lam = np.random.beta(alpha, alpha)

    image_h, image_w = images.shape[2:]
    cx = np.random.uniform(0, image_w)
    cy = np.random.uniform(0, image_h)
    w = image_w * np.sqrt(1 - lam)
    h = image_h * np.sqrt(1 - lam)
    x0 = int(np.round(max(cx - w / 2, 0)))
    x1 = int(np.round(min(cx + w / 2, image_w)))
    y0 = int(np.round(max(cy - h / 2, 0)))
    y1 = int(np.round(min(cy + h / 2, image_h)))

    images[:, :, y0:y1, x0:x1] = shuffled_images[:, :, y0:y1, x0:x1]

    return {
        'image': images,
        'target': (targets, shuffled_targets, lam)
    }

def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

In [26]:
def train_and_evaluate(model, optimizer, scheduler, train_loader, valid_loader, fold, trial_number):
    criterion = nn.BCEWithLogitsLoss()
    best_score = 0
    best_model = None
    patience = CONFIG['patience']
    patience_counter = 0

    for epoch in range(CONFIG['epochs']):

        model.train()
        train_loss = 0
        train_predictions = []
        train_targets = []

        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{CONFIG['epochs']} - Training"):
            images = batch['image'].to(CONFIG['device'])
            targets = batch['target'].to(CONFIG['device'])

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, targets.unsqueeze(1).float())

            loss.backward()
            optimizer.step()

            train_loss += loss.item()
            train_predictions.extend(torch.sigmoid(outputs).cpu().detach().numpy())
            train_targets.extend(targets.cpu().numpy())

        train_loss /= len(train_loader)
        train_score = custom_metric_binary(np.array(train_targets), np.array(train_predictions))


        model.eval()
        valid_loss = 0
        valid_predictions = []
        valid_targets = []

        with torch.no_grad():
            for batch in tqdm(valid_loader, desc=f"Epoch {epoch+1}/{CONFIG['epochs']} - Validation"):
                images = batch['image'].to(CONFIG['device'])
                targets = batch['target'].to(CONFIG['device'])

                outputs = model(images)
                loss = criterion(outputs, targets.unsqueeze(1).float())

                valid_loss += loss.item()
                valid_predictions.extend(torch.sigmoid(outputs).cpu().numpy())
                valid_targets.extend(targets.cpu().numpy())

        valid_loss /= len(valid_loader)
        valid_score = custom_metric_binary(np.array(valid_targets), np.array(valid_predictions))

        print(f"Epoch {epoch+1}/{CONFIG['epochs']} - Train Loss: {train_loss:.4f}, Train Score: {train_score:.4f}, Valid Loss: {valid_loss:.4f}, Valid Score: {valid_score:.4f}")

        if valid_score > best_score:
            best_score = valid_score
            best_model = copy.deepcopy(model.state_dict())
            patience_counter = 0
            print(f"New best score: {best_score:.4f}")
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"Early stopping triggered after {epoch+1} epochs")
            break

        scheduler.step()


    model_dir = f'/content/model'
    os.makedirs(model_dir, exist_ok=True)
    model_path = os.path.join(model_dir, f'384_model_fold_{fold}.pth')
    torch.save(best_model, model_path)
    print(f"Best model for fold {fold} saved with score: {best_score:.4f} at {model_path}")

    return best_score


In [27]:
from sklearn.metrics import roc_auc_score
def custom_metric_binary(y_true, y_pred):

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)


    if y_pred.ndim == 2 and y_pred.shape[1] == 1:
        y_pred = y_pred.squeeze()


    elif y_pred.ndim == 2 and y_pred.shape[1] == 2:
        y_pred = y_pred[:, 1]


    assert y_pred.ndim == 1, "Predictions should be 1-dimensional after processing"

    min_tpr = 0.80
    max_fpr = 1 - min_tpr

    v_gt = 1 - y_true
    v_pred = 1 - y_pred

    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)

    return partial_auc

In [24]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import numpy as np
import copy
from collections import defaultdict
import time
import gc
import os


def objective(trial):
    trial_number = trial.number
    CONFIG.update({
        "learning_rate": trial.suggest_loguniform("learning_rate", 1e-5, 1e-3),
        "min_lr": trial.suggest_loguniform("min_lr", 1e-7, 1e-5),
        "weight_decay": trial.suggest_loguniform("weight_decay", 1e-6, 1e-4),
        "n_accumulate": trial.suggest_int("n_accumulate", 1, 6),
        "patience": trial.suggest_int("patience", 3, 10),
        "dropout_rate": trial.suggest_uniform("dropout_rate", 0.1, 0.5),
        "cutmix_prob": trial.suggest_uniform("cutmix_prob", 0.0, 1.0)
    })
    print(CONFIG)

    fold_scores = []
    for fold in range(CONFIG['n_fold']):
        print(f"===== Fold: {fold} =====")

        train_dataset = ISICDataset(balanced_df[balanced_df.fold != fold], train_hdf, transforms=data_transforms["train"])
        valid_dataset = ISICDataset(balanced_df[balanced_df.fold == fold], train_hdf, transforms=data_transforms["valid"])

        train_loader = DataLoader(train_dataset, batch_size=CONFIG['train_batch_size'],
                                  shuffle=True, num_workers=2, pin_memory=True)
        valid_loader = DataLoader(valid_dataset, batch_size=CONFIG['valid_batch_size'],
                                  shuffle=False, num_workers=2, pin_memory=True)

        model = ISICModel(CONFIG['model_name'], num_classes=CONFIG['num_classes'],
                          pretrained=True, dropout_rate=CONFIG['dropout_rate'])
        model.to(CONFIG['device'])

        optimizer = optim.AdamW(model.parameters(), lr=CONFIG['learning_rate'],
                                weight_decay=CONFIG['weight_decay'])
        scheduler = fetch_scheduler(optimizer)

        best_score = train_and_evaluate(model, optimizer, scheduler, train_loader, valid_loader, fold, trial_number)
        fold_scores.append(best_score)

    mean_score = np.mean(fold_scores)
    print(f"Trial {trial_number} completed. Mean score: {mean_score:.4f}")
    return mean_score


In [28]:
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=1)

print("Best trial:")
trial = study.best_trial
print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))


best_trial_number = study.best_trial.number
print(f"The best models are saved in the directory: models/trial_{best_trial_number}/")


[I 2024-09-11 14:38:10,093] A new study created in memory with name: no-name-d34cf092-fb41-4718-9438-acac0569ece0


{'seed': 42, 'epochs': 30, 'img_size': 384, 'model_name': 'tf_efficientnetv2_b1.in1k', 'train_batch_size': 64, 'valid_batch_size': 128, 'learning_rate': 0.00012248652806198703, 'scheduler': 'CosineAnnealingWarmRestarts', 'min_lr': 1.6978183451778045e-06, 'T_max': 500, 'weight_decay': 4.209017949562916e-05, 'n_fold': 5, 'n_accumulate': 2, 'device': device(type='cuda', index=0), 'num_classes': 1, 'T_0': 10, 'T_mult': 1, 'patience': 7, 'dropout_rate': 0.34005734074156396, 'cutmix_prob': 0.09923012058552605}
===== Fold: 0 =====


Epoch 1/30 - Validation: 100%|██████████| 8/8 [00:03<00:00,  2.20it/s]


Epoch 1/30 - Train Loss: 0.3817, Train Score: 0.0270, Valid Loss: 0.2488, Valid Score: 0.1164
New best score: 0.1164


Epoch 2/30 - Validation: 100%|██████████| 8/8 [00:03<00:00,  2.20it/s]


...

Epoch 10/30 - Train Loss: 0.1597, Train Score: 0.1429, Valid Loss: 0.1450, Valid Score: 0.1506


Epoch 11/30 - Validation: 100%|██████████| 8/8 [00:03<00:00,  2.29it/s]


Epoch 11/30 - Train Loss: 0.1595, Train Score: 0.1424, Valid Loss: 0.1433, Valid Score: 0.1517


Epoch 12/30 - Validation: 100%|██████████| 8/8 [00:03<00:00,  2.27it/s]
[I 2024-09-11 15:37:24,683] Trial 0 finished with value: 0.15252951885988159 and parameters: {'learning_rate': 0.00012248652806198703, 'min_lr': 1.6978183451778045e-06, 'weight_decay': 4.209017949562916e-05, 'n_accumulate': 2, 'patience': 7, 'dropout_rate': 0.34005734074156396, 'cutmix_prob': 0.09923012058552605}. Best is trial 0 with value: 0.15252951885988159.


Epoch 12/30 - Train Loss: 0.1555, Train Score: 0.1499, Valid Loss: 0.1483, Valid Score: 0.1539
Early stopping triggered after 12 epochs
Best model for fold 4 saved with score: 0.1582 at /content/model/384_model_fold_4.pth
Trial 0 completed. Mean score: 0.1525
Best trial:
  Value:  0.15252951885988159
  Params: 
    learning_rate: 0.00012248652806198703
    min_lr: 1.6978183451778045e-06
    weight_decay: 4.209017949562916e-05
    n_accumulate: 2
    patience: 7
    dropout_rate: 0.34005734074156396
    cutmix_prob: 0.09923012058552605
The best models are saved in the directory: models/trial_0/


In [29]:
train_meta = pd.read_csv('https://raw.githubusercontent.com/Chienstartup/ISIC_2024/main/meta_data/df_train_filtered.csv')

In [30]:
import torch
from torch.utils.data import DataLoader
import pandas as pd
import numpy as np
from tqdm import tqdm
import torch.optim as optim
from sklearn.metrics import roc_auc_score
import os
import h5py
from io import BytesIO
from PIL import Image

def custom_metric_binary(y_true, y_pred):
    y_hat = y_pred if y_pred.ndim == 1 else y_pred[:, 1]
    y_true_binary = y_true

    min_tpr = 0.80
    max_fpr = 1 - min_tpr

    v_gt = 1 - y_true_binary
    v_pred = 1 - y_hat

    partial_auc_scaled = roc_auc_score(v_gt, v_pred, max_fpr=max_fpr)
    partial_auc = 0.5 * max_fpr**2 + (max_fpr - 0.5 * max_fpr**2) / (1.0 - 0.5) * (partial_auc_scaled - 0.5)

    return partial_auc

def load_model(model_path, device):
    model = ISICModel(CONFIG['model_name'], num_classes=1, pretrained=False)
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    return model

@torch.no_grad()
def predict(models, dataloader, device):
    predictions = []
    for batch in tqdm(dataloader):
        images = batch['image'].to(device)
        outputs = [model(images) for model in models]
        probs = torch.mean(torch.stack([torch.sigmoid(output).squeeze() for output in outputs]), dim=0)
        predictions.extend(probs.cpu().numpy().tolist())
    return predictions

def main():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")


    if 'fold' not in train_meta.columns:
        print("Error: 'fold' column not found in the dataframe.")
        return

    # load hdf file
    with h5py.File('/content/sample_5k.hdf5', 'r') as train_hdf:
        oof_predictions = np.zeros(len(train_meta))

        # set the model path
        model_paths = [
            '/content/model/384_model_fold_0.pth',
            '/content/model/384_model_fold_1.pth',
            '/content/model/384_model_fold_2.pth',
            '/content/model/384_model_fold_3.pth',
            '/content/model/384_model_fold_4.pth',

        ]

        # loading all models
        models = []
        for path in model_paths:
            if os.path.exists(path):
                model = load_model(path, device)
                models.append(model)
                print(f"Loaded pre-trained model: {path}")
            else:
                print(f"Pre-trained model not found: {path}. Skipping...")

        if not models:
            print("No models loaded. Exiting...")
            return

        # oof prediction
        dataset = ISICDataset(train_meta, train_hdf, transforms=data_transforms['valid'])
        dataloader = DataLoader(dataset, batch_size=CONFIG['valid_batch_size'], shuffle=False, num_workers=2)
        oof_predictions = predict(models, dataloader, device)

    oof_df = pd.DataFrame({
        'isic_id': train_meta['isic_id'].values,
        'target': train_meta['target'].values,
        'oof_prediction': oof_predictions,
        'fold': train_meta['fold'].values
    })

    oof_df.to_csv("/content/model/384_v2b1_21k.csv", index=False)
    print("OOF Predictions saved to 384_v2b1_21k.csv")

    oof_auc = roc_auc_score(oof_df['target'], oof_df['oof_prediction'])
    print(f"Overall OOF AUC: {oof_auc:.4f}")

    custom_metric = custom_metric_binary(oof_df['target'].values, oof_df['oof_prediction'].values)
    print(f"Overall OOF Custom Metric: {custom_metric:.4f}")

if __name__ == "__main__":
    main()

Using device: cuda
Loaded pre-trained model: /content/model/384_model_fold_0.pth
Loaded pre-trained model: /content/model/384_model_fold_1.pth
Loaded pre-trained model: /content/model/384_model_fold_2.pth
Loaded pre-trained model: /content/model/384_model_fold_3.pth
Loaded pre-trained model: /content/model/384_model_fold_4.pth


100%|██████████| 40/40 [01:08<00:00,  1.72s/it]

OOF Predictions saved to 384_v2b1_21k.csv
Overall OOF AUC: 0.9767
Overall OOF Custom Metric: 0.1836
